# Mann-Kendall Trend Test Example (MW -103s)

In [83]:
import numpy as np
from scipy.stats import norm, mstats
from scipy.integrate import quad
import statistics as stats

In [84]:
def mann_kendall(x, alpha=0.05):
    n = len(x)
    coeff_var = stats.stdev(x) / stats.mean(x)
    
    # Calculate S #
    S = 0
    for k in range(n - 1):
        for j in range(k + 1, n):
            S += np.sign(x[j] - x[k])
            
    # Calculate the unique data #
    unique_x = np.unique(x)
    g = len(unique_x)
    
    # Calculate the variance of S #
    if n == g:
        var_S = (n * (n - 1) * (2 * n + 5)) / 18
    else:
        tp = np.zeros(unique_x.shape)
        for i in range(g):
            tp[i] = sum(unique_x[i] == x)
        var_S = (n * (n - 1) * (2 * n + 5) + np.sum(tp * (tp -1) * (2 * tp + 5))) / 18
        
    # Calculate Z statistic #
    if S > 0:
        Z = (S - 1) / np.sqrt(var_S)
    elif S == 0:
        Z = 0
    else:
        Z = (S + 1) / np.sqrt(var_S)
        
    # Calculate the p-value #
    p = 2 * (1 - norm.cdf(abs(Z)))
    h = abs(Z) > norm.ppf(1 - alpha / 2)
    
    if (Z < 0) and h:
        trend = 'DECREASING'
    elif (Z > 0) and h:
        trend = 'INCREASING'
    else:
        trend = 'NO TREND'
        
    return trend, h, p, Z, S, coeff_var

In [85]:
# Calculate the Confidence Factor #
def normal_probability_density(x):
    constant = 1.0 / np.sqrt(2 * np.pi)
    return (constant * np.exp((-x**2) / 2.0))

In [86]:
data = [78, 76, 74, 73, 78, 70, 75, 66, 52, 64, 69, 73, 69, 71, 72, 72, 69, 58, 62, 51, 51, 53, 58, 65, 75, 72, 71, 72, 63, 64, 68, 71, 60, 65, 77, 76, 85, 83, 88, 89]

In [87]:
test_trend, h, p, Z, S, CV = mann_kendall(data, alpha=0.05)
last_eight, h_8, p_8, Z_8, S_8, CV_8 = mann_kendall(data[-8:], alpha=0.05)
test_cf, _ = quad(normal_probability_density, np.NINF, Z)
last_eight_cf, _ = quad(normal_probability_density, np.NINF, Z_8)

In [88]:
print("COMPLETE TEST : Trend: {}, S: {}, Coefficient of Variation: {:.2f}, Confidence Factor: {:.1%}".format(test_trend, S, CV, test_cf))
print("LAST EIGHT : Trend: {}, S: {}, Coefficient of Variation: {:.2f}, Confidence Factor: {:.1%}".format(last_eight, S_8, CV_8, last_eight_cf))

COMPLETE TEST : Trend: NO TREND, S: 34, Coefficient of Variation: 0.13, Confidence Factor: 64.9%
LAST EIGHT : Trend: INCREASING, S: 24, Coefficient of Variation: 0.14, Confidence Factor: 99.8%
